## Adım 1: Setup, Veri Yükleme ve Son Özellik Setini Hazırlama
Feature Engineering dosyasında oluşturduğumuz ve en önemli 5 özellikten oluşan o küçültülmüş feature setini (özellik setini) bu dosyaya getirmemiz gerekiyor.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV 
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error, r2_score

# --- 1. VERİ YÜKLEME ---
print("⏳ Veri yükleniyor...")
try:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='utf-8')
except Exception:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='latin1')

# Temizlik: Log dönüşümü için 0 olan değerleri atıyoruz
df = df[(df['view_count'] > 0) & (df['likes'] > 0)].copy()

# --- 2. EKSİK ÖZELLİKLERİ TEKRAR TÜRETME (RE-ENGINEERING) ---
# Bu özellikler ham CSV'de olmadığı için burada tekrar yaratmalıyız.

# A) Başlık Uzunluğu
df['title_length'] = df['title'].str.len()

# B) Etiket Sayısı
df['tag_count'] = df['tags'].apply(lambda x: 0 if x == '[none]' else len(str(x).split('|')))

# --- 3. LOG DÖNÜŞÜMLERİ ---
df['log_view_count'] = np.log1p(df['view_count'])
df['log_likes'] = np.log1p(df['likes'])
df['log_comment_count'] = np.log1p(df['comment_count'])

# --- 4. NİHAİ ÖZELLİK SETİNİ SEÇME ---
# Feature Selection'da belirlediğimiz en iyi 5 özellik
selected_features = [
    'log_likes', 
    'log_comment_count', 
    'dislikes', 
    'title_length', 
    'tag_count'
]

# Hedef ve Özellikleri Ayırma
X = df[selected_features]
y = df['log_view_count']

# --- 5. VERİ BÖLME (TRAIN / VAL / TEST) ---
# Önce Test'i (%20) ayır
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Sonra Validation'ı (%20) ayır
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42
)

print("✅ Veri Hazırlandı ve Eksik Sütunlar Türetildi.")
print(f"Eğitim Seti: {X_train.shape[0]} satır")
print(f"Kullanılan Özellikler: {X.columns.tolist()}")

⏳ Veri yükleniyor...
✅ Veri Hazırlandı ve Eksik Sütunlar Türetildi.
Eğitim Seti: 171024 satır
Kullanılan Özellikler: ['log_likes', 'log_comment_count', 'dislikes', 'title_length', 'tag_count']


###  Veri Hazırlığı ve Final Özellik Entegrasyonu
 `3_feature_engineering.ipynb` dosyasından türetilen (`title_length`, `tag_count`) gibi özellikleri bu dosyaya başarıyla yeniden entegre ederek önceki `KeyError` hatalarını gidermiştir. Modelin eğitileceği tüm özellikler ve hedef değişken (`view_count`), doğrusal ilişkiyi sağlamak amacıyla her iki tarafa da **Logaritmik Dönüşüm**den geçirilmiştir. Nihai aşamada, Feature Selection ile belirlenen **5 en güçlü özellik** (Etkileşim ve İçerik Stratejisi) kullanılarak veri, **171.024 satır** ile Train/Validation/Test setlerine ayrılmıştır.

## Adım 2: Model Eğitimi ve Optimizasyon


###  Algoritma Seçimi:

Final model olarak **XGBoost Regressor** tercih edilmiştir. Bu seçim, Baseline kıyaslamasında en yüksek skoru veren **ağaç tabanlı** (Random Forest: 0.6509) model ailesine dayanmaktadır. XGBoost, Random Forest'tan daha hızlı ve daha doğru tahminler yapabilen, endüstri standardı bir algoritmadır. Hedefimiz, bu güçlü algoritmayı kullanarak **0.6509** olan Baseline skoru aşmaktır.

### Hiperparametre Optimizasyonu:

In [4]:
# --- HİPERPARAMETRE OPTİMİZASYONU ---
print("⏳ GridSearch başlatılıyor... Bu işlem birkaç dakika sürebilir.")

# 1. Kıyaslama Yapılacak Parametreler
# Optimizasyonun hızını artırmak için basit bir aralık seçiyoruz
param_grid = {
    'n_estimators': [100, 200], # Kaç ağaç kullanılacak?
    'max_depth': [3, 5],        # Ağacın derinliği (Komplekslik)
    'learning_rate': [0.05, 0.1] # Öğrenme hızı
}

# 2. XGBoost Modeli
xgb_model = XGBRegressor(random_state=42)

# 3. GridSearchCV (En iyi parametreyi bul)
# Cross-Validation (cv=3) ile denemeler yaparız. n_jobs=-1 tüm çekirdekleri kullanır.
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           scoring='r2', cv=3, verbose=1, n_jobs=-1)

# 4. Eğitimi Başlat (Validation/Train verisini kullanıyoruz)
# Optimizasyonu, Train ve Validation birleşik set üzerinde yapıyoruz.
grid_search.fit(X_train_val, y_train_val)

print("\n✅ Optimizasyon Tamamlandı.")
print(f"En İyi Parametreler: {grid_search.best_params_}")

# 5. En İyi Modeli Kaydet ve Skorla
best_xgb = grid_search.best_estimator_

# Sadece nihai skoru görmek için Tahmin yapalım (Validation Seti üzerinde)
y_val_pred = best_xgb.predict(X_val)
r2_optimized = r2_score(y_val, y_val_pred)

print(f"Optimizasyon Sonrası R2 Skoru (Validation Seti): {r2_optimized:.4f}")

⏳ GridSearch başlatılıyor... Bu işlem birkaç dakika sürebilir.
Fitting 3 folds for each of 8 candidates, totalling 24 fits

✅ Optimizasyon Tamamlandı.
En İyi Parametreler: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Optimizasyon Sonrası R2 Skoru (Validation Seti): 0.7802


### Optimizasyon Sonuç ve Raporu

**1. En İyi Parametreler:** GridSearch, **learning_rate: 0.1, max_depth: 5, n_estimators: 200** parametrelerini en iyi kombinasyon olarak belirlemiştir.

**2. R2 Skoru (Nihai Proje Skoru):** **0.7802**

**3. Çıkarım:** Optimizasyon sonucunda elde edilen R2 skoru (**0.7802**), Baseline skoru olan **0.6509**'u anlamlı bir farkla aşmıştır. Bu durum, XGBoost algoritması ve seçilen 5 kritik özelliğin (Etkileşim Metrikleri + İçerik Uzunluğu) YouTube verisindeki karmaşık desenleri çözmede başarılı olduğunu kanıtlamaktadır. Modelimiz artık **%78** doğrulukla izlenmeleri tahmin edebilmektedir.